<a href="https://colab.research.google.com/github/AU2302DM-TeamKYJ/TeamProject-SteamGameRecommander/blob/main/Lab/UserCrawler_SHY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import urllib.request
import json
from pandas import json_normalize
from urllib.parse import urlencode, quote_plus, unquote

In [18]:
# API 호출을 위한 설정 준비 작업
api_link = 'http://api.steampowered.com/ISteamUser/GetFriendList/v0001/'
api_key = '...'
steamid = '76561198003081163' # 스트리머 우왁굳
queryParams = '?' + urlencode({quote_plus('key') : api_key,
                               quote_plus('steamid') : steamid,
                               quote_plus('relationship') : 'friend'
                               })
url = api_link + queryParams

In [19]:
#데이터를 요청하고 변환합니다.
text = urllib.request.urlopen(url).read().decode('utf-8')
json_return = json.loads(text)

In [20]:
print(json_return)

{'friendslist': {'friends': [{'steamid': '76561197960421647', 'relationship': 'friend', 'friend_since': 1490008534}, {'steamid': '76561197966062591', 'relationship': 'friend', 'friend_since': 1595669725}, {'steamid': '76561197967479980', 'relationship': 'friend', 'friend_since': 1502453076}, {'steamid': '76561197971565071', 'relationship': 'friend', 'friend_since': 1423219650}, {'steamid': '76561197972650406', 'relationship': 'friend', 'friend_since': 1431962639}, {'steamid': '76561197973315893', 'relationship': 'friend', 'friend_since': 1461849261}, {'steamid': '76561197978049807', 'relationship': 'friend', 'friend_since': 1459594769}, {'steamid': '76561197986317848', 'relationship': 'friend', 'friend_since': 1502453066}, {'steamid': '76561197987498968', 'relationship': 'friend', 'friend_since': 1434712783}, {'steamid': '76561197987573026', 'relationship': 'friend', 'friend_since': 1635075281}, {'steamid': '76561197988813283', 'relationship': 'friend', 'friend_since': 1451056439}, {'s

In [21]:
friends_list = {}
queue_user_to_scan = []

# 반환된 JSON에서 친구 목록만 가져오기
friends_data = json_return.get('friendslist').get('friends')

# 새 User의 친구 목록으로 작업
for friend in friends_data:
    if friend.get('steamid') not in friends_list:         # 해당 친구가 이미 스캔된 유저가 아닌 경우
        queue_user_to_scan.append(friend.get('steamid'))    # 해당 친구를 스캔 대상 큐에 추가
        friends_list[friend.get('steamid')] = 1             # 해당 친구를 친구 수집 목록에 추가
    else:                                                 # 해당 친구가 이미 스캔된 유저인 경우
        continue                                            # 스킵 후 계속 진행

In [81]:
print(friends_list)
print(queue_user_to_scan)
print(len(friends_list), len(queue_user_to_scan))

{'76561197960421647': '1', '76561197966062591': '1', '76561197967479980': '1', '76561197971565071': '1', '76561197972650406': '1', '76561197973315893': '1', '76561197978049807': '1', '76561197986317848': '1', '76561197987498968': '1', '76561197987573026': '1', '76561197988813283': '1', '76561197989996681': '1', '76561197991732696': '1', '76561197992224881': '1', '76561197993437482': '1', '76561197996328386': '1', '76561197996477270': '1', '76561197997302450': '1', '76561197998917125': '1', '76561197998966578': '1', '76561197999552510': '1', '76561198000488531': '1', '76561198001155559': '1', '76561198002819350': '1', '76561198003432796': '1', '76561198003440990': '1', '76561198003929504': '1', '76561198003941592': '1', '76561198004024187': '1', '76561198004447623': '1', '76561198004543392': '1', '76561198004954375': '1', '76561198004967330': '1', '76561198005233788': '1', '76561198005437683': '1', '76561198005475569': '1', '76561198006037162': '1', '76561198006218291': '1', '7656119800

In [71]:
# 큐에서 스캔 대상자 추출 후 쿼리 작성
user_to_scan = queue_user_to_scan.pop(0)
queryParams = '?' + urlencode({quote_plus('key') : api_key,
                               quote_plus('steamid') : user_to_scan,
                               quote_plus('relationship') : 'friend'
                               })
url = api_link + queryParams

# 데이터를 요청하고 변환 : 스캔한 사용자 프로필이 Public이 아닌 경우 401 에러에 대한 예외 처리
try:
    text = urllib.request.urlopen(url).read().decode('utf-8')
except:
    text = '{}'
json_return = json.loads(text)

# 데이터 해석 작업
if json_return != {}:
    # 반환된 JSON에서 친구 목록만 가져오기
    friends_data = json_return.get('friendslist').get('friends')

    # 새 User의 친구 목록으로 작업
    for friend in friends_data:
        if friend.get('steamid') not in friends_list:         # 해당 친구가 이미 스캔된 유저가 아닌 경우
            queue_user_to_scan.append(friend.get('steamid'))    # 해당 친구를 스캔 대상 큐에 추가
            friends_list[friend.get('steamid')] = 1             # 해당 친구를 친구 수집 목록에 추가
        else:                                                 # 해당 친구가 이미 스캔된 유저인 경우
            continue                                            # 스킵 후 계속 진행

In [75]:
# 추출한 계정 값들이 유니크한지 검증
my_friends = friends_list.keys()
list_my_friends = list(set(my_friends))
print(list_my_friends)
print(len(list_my_friends))

['76561198302881020', '76561197995383338', '76561198000514549', '76561198316421119', '76561198112143903', '76561198020398851', '76561198047999009', '76561198068740421', '76561198024810116', '76561198078586073', '76561198141509145', '76561198381419449', '76561198009891312', '76561198009849040', '76561198003941592', '76561198109146962', '76561198037528586', '76561198044284553', '76561198254590677', '76561198122834714', '76561197960421647', '76561198432737917', '76561198059464928', '76561198100765866', '76561197998917125', '76561198152240484', '76561198103256307', '76561198067964167', '76561198250836314', '76561198010209361', '76561198128023412', '76561198016127992', '76561197969782654', '76561198225934124', '76561198037808929', '76561199016229331', '76561198005437683', '76561198271083824', '76561198260390003', '76561198047238837', '76561197988920493', '76561198115634307', '76561198157910326', '76561198166610803', '76561198013471132', '76561198327679056', '76561198410333011', '76561198118

In [76]:
# 지속 추출을 위한 리스트&딕셔너리 임시 저장
import csv

with open("/content/drive/MyDrive/Colab Notebooks/DM2302/Project/Friends.csv", 'w') as file:
    writer = csv.writer(file)
    for k, v in friends_list.items():
        writer.writerow([k, v])
with open('/content/drive/MyDrive/Colab Notebooks/DM2302/Project/Queue.csv','w',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(queue_user_to_scan)

In [77]:
# 지속 추출 테스트용 리스트&딕셔너리 초기화

friends_list = {}
queue_user_to_scan = []

In [80]:
# 지속 추출을 위한 리스트&딕셔너리 불러오기
import csv

friends_list = {}
queue_user_to_scan = []

with open("/content/drive/MyDrive/Colab Notebooks/DM2302/Project/Friends.csv", 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        friends_list[row[0]] = row[1]
with open('/content/drive/MyDrive/Colab Notebooks/DM2302/Project/Queue.csv','r',newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        queue_user_to_scan = row